In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.0+cu113
     |████████████████████████████████| 7.9 MB 46.2 MB/s 
     |████████████████████████████████| 2.4 MB 52.0 MB/s 
     |████████████████████████████████| 3.5 MB 45.4 MB/s 


In [ ]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
#%cd /content/drive/MyDrive/GNN_DATA/ADNI/ALL_NETWORKS
%cd /content/drive/MyDrive/ALL_NETWORKS

for i in range(1, 110):
    if os.path.isfile('AD_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('AD_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp=torch.div(tmp,tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      deg = degree(data.edge_index[0], data.num_nodes)
      x = []
      for i in deg:
        x.extend([i])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

for i in range(1, 110):
    if os.path.isfile('CN_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('CN_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp=torch.div(tmp, tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      data = from_networkx(G)
      deg = degree(data.edge_index[0], data.num_nodes)
      x = []
      for i in deg:
        x.extend([i])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/10M6M6of9sCS0_CWq5d9EF6LhBCK9klNf/ALL_NETWORKS


In [ ]:
%cd /content/drive/MyDrive
pos = pd.read_csv("node_aal90.csv", header = None)

/content/drive/MyDrive


In [ ]:
position1 = []
for i in pos[0]:
  position1.extend([i])
position1 = np.array(position1).reshape(90,1)

In [ ]:
position2 = []
for i in pos[1]:
  position2.extend([i])
position2 = np.array(position2).reshape(90,1)

In [ ]:
position3 = []
for i in pos[2]:
  position3.extend([i])
position3 = np.array(position3).reshape(90,1)

In [ ]:
position = np.concatenate((position1, position2, position3), axis = 1) 

In [ ]:
position

In [ ]:
for i in graphlist:
  i.pos = torch.FloatTensor(position)

In [ ]:
from torch_geometric.loader import DataLoader
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=True)

In [ ]:
dataset[0].edge_index

tensor([[ 0,  0,  0,  ..., 89, 89, 89],
        [ 1,  2,  3,  ..., 83, 85, 87]])

In [ ]:
import torch
from torch_geometric.utils import normalized_cut


def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    #print(edge_attr)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

def normalized_cut_3d(edge_index, pos):
    row, col= edge_index
    print(pos[col], "row")
    edge_attr = torch.norm(pos[row] - pos[col], p=3, dim = 1)
    print(edge_attr)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0)) 

stmp = normalized_cut_3d(dataset[0].edge_index, dataset[0].pos)

tensor([[ 41.3700,  -8.2100,  52.0900],
        [-18.4500,  34.8100,  42.2000],
        [ 21.9000,  31.1200,  43.8200],
        ...,
        [ 48.2500,  14.7500, -16.8600],
        [ 57.4700, -37.2300,  -1.4700],
        [ 44.2200,  14.5500, -32.2300]]) row
tensor([80.0209, 42.2261, 64.8075,  ..., 45.8713, 21.0683, 45.9101])


In [ ]:
import torch
from torch_geometric.utils import normalized_cut


def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops


class SpatialGraphConv(MessagePassing):
    def __init__(self, coors, in_channels, out_channels, hidden_size, dropout=0):
        """
        coors - dimension of positional descriptors (e.g. 2 for 2D images)
        in_channels - number of the input channels (node features)
        out_channels - number of the output channels (node features)
        hidden_size - number of the inner convolutions
        dropout - dropout rate after the layer
        """
        super(SpatialGraphConv, self).__init__(aggr='add')
        self.dropout = dropout
        self.lin_in = torch.nn.Linear(coors, hidden_size * in_channels)
        self.lin_out = torch.nn.Linear(hidden_size * in_channels, out_channels)
        self.in_channels = in_channels

    def forward(self, x, pos, edge_index):
        """
        x - feature matrix of the whole graph [num_nodes, label_dim]
        pos - node position matrix [num_nodes, coors]
        edge_index - graph connectivity [2, num_edges]
        """
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))  # num_edges = num_edges + num_nodes

        return self.propagate(edge_index=edge_index, x=x, pos=pos, aggr='add')  # [N, out_channels, label_dim]

    def message(self, pos_i, pos_j, x_j):
        """
        pos_i [num_edges, coors]
        pos_j [num_edges, coors]
        x_j [num_edges, label_dim]
        """

        relative_pos = pos_j - pos_i  # [n_edges, hidden_size * in_channels]
        spatial_scaling = F.relu(self.lin_in(relative_pos))  # [n_edges, hidden_size * in_channels]

        n_edges = spatial_scaling.size(0)
        # [n_edges, in_channels, ...] * [n_edges, in_channels, 1]
        result = spatial_scaling.reshape(n_edges, self.in_channels, -1) * x_j.unsqueeze(-1)
        return result.view(n_edges, -1)

    def update(self, aggr_out):
        """
        aggr_out [num_nodes, label_dim, out_channels]
        """
        aggr_out = self.lin_out(aggr_out)  # [num_nodes, label_dim, out_features]
        aggr_out = F.relu(aggr_out)
        aggr_out = F.dropout(aggr_out, p=self.dropout, training=self.training)

        return aggr_out

In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import graclus, max_pool, global_mean_pool

class SGCN(torch.nn.Module):
    def __init__(self, dim_coor, out_dim, input_features,
                 layers_num, model_dim, out_channels_1, dropout,
                 use_cluster_pooling):
        super(SGCN, self).__init__()
        self.layers_num = layers_num
        self.use_cluster_pooling = use_cluster_pooling

        self.conv_layers = [SpatialGraphConv(coors=dim_coor,
                                             in_channels=input_features,
                                             out_channels=model_dim,
                                             hidden_size=out_channels_1,
                                             dropout=dropout)] + \
                           [SpatialGraphConv(coors=dim_coor,
                                             in_channels=model_dim,
                                             out_channels=model_dim,
                                             hidden_size=out_channels_1,
                                             dropout=dropout) for _ in range(layers_num - 1)]

        self.conv_layers = torch.nn.ModuleList(self.conv_layers)

        self.fc1 = torch.nn.Linear(model_dim, out_dim)

    def forward(self, data):
        for i in range(self.layers_num):
            data.x = self.conv_layers[i](data.x, data.pos, data.edge_index)

            if self.use_cluster_pooling:
                weight = normalized_cut_2d(data.edge_index, data.pos)
                cluster = graclus(data.edge_index, weight, data.x.size(0))
                data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        data.x = global_mean_pool(data.x, data.batch)
        x = self.fc1(data.x)

        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sig=torch.nn.Sigmoid()
model = SGCN(dim_coor=3,
             out_dim=1,
             input_features=1,
             layers_num=3,
             model_dim=16,
             out_channels_1=64,
             dropout=0.3,
             use_cluster_pooling=True).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
#rotation_0 = T.RandomRotate(degrees=180, axis=0)
#rotation_1 = T.RandomRotate(degrees=180, axis=1)
#rotation_2 = T.RandomRotate(degrees=180, axis=2)


def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = output.view(-1)
        loss = criterion(output, data.y.float()) 
        #loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data).max(dim=1)[1]
        pred = sig(out)>0.5
        #pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


train_acc_array = []
test_acc_array = []

for epoch in range(1, 100):
    loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)

    train_acc_array.append(train_acc)
    test_acc_array.append(test_acc)

    print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.format(epoch, loss, train_acc, test_acc))

Epoch: 001, Loss: 12489700.66667, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 002, Loss: 310118.78385, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 003, Loss: 55181.47426, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 004, Loss: 2035.26542, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 005, Loss: 0.70037, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 006, Loss: 0.70030, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 007, Loss: 0.70001, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 008, Loss: 0.69981, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 009, Loss: 0.69958, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 010, Loss: 0.69933, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 011, Loss: 0.69910, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 012, Loss: 0.69881, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 013, Loss: 0.69860, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 014, Loss: 0.69834, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: 015, Loss: 0.69807, Train Acc: 0.55333, Test Acc: 0.40385
Epoch: